In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [135]:
data_path = "/Users/raghavatal/Developer/Data/Petfinder_DataMiningProject/petfinder-adoption-prediction"
train_df = pd.read_csv(data_path+'/train.csv')
train_df.shape

(14993, 24)

### Replace Missing values in Name field with 'No Name'

In [145]:
train_copy_df = train_df

In [15]:
def countNanInColumn(df,col):
    count = df.loc[df[col].isnull(),col].shape[0]
    return count

In [52]:
def replaceNanWithValue(df,col,value):
    count = countNanInColumn(df,col)
    df.loc[df[col].isnull(),col] = value
    return count

In [146]:
#Replace blank values in Name with 'No Name'
#train_copy_df.lo``c[train_copy_df['Name'].isnull(),'Name'] = 'No Name'
count = replaceNanWithValue(train_copy_df,'Name','No Name')
print('Values Replaced: {}'.format(count))

Values Replaced: 0


### Add Color Count

In [147]:
train_copy_df['NumColors'] = train_copy_df.loc[:,'Color1':'Color3'].apply(lambda row: bool(row.Color1)
                                                         + bool(row.Color2) 
                                                         + bool(row.Color3),axis=1)
train_copy_df.shape

(14993, 27)

### Add IsMixedBreed flag

In [148]:
train_copy_df['IsMixedBreed'] = train_copy_df.loc[:,'Breed1':'Breed2'].apply(lambda row: bool(row.Breed1)*bool(row.Breed2),axis=1)
train_copy_df.shape

(14993, 27)

### Add name frequency column

In [149]:
nameFrequency_df = pd.DataFrame(train_copy_df['Name'].value_counts())
#nameFrequency_df.reset_index()
train_copy_df['NameFrequency'] = nameFrequency_df.loc[train_copy_df['Name'],'Name'].values
train_copy_df.shape

(14993, 27)

### Reading sentiment score file from Tobel and merging with main dataframe

In [144]:
pd.merge?

In [151]:
pd.set_option('display.max_columns', 500)
sentiment_df = pd.read_csv(data_path+'/Processed/sentiment.csv')
sentiment_df.set_index('PetID')
(left ,right, key)  = (train_copy_df ,sentiment_df[['PetID','Magnitude','Score']],'PetID')
train_copy_df=pd.merge(left, right, on=key,how='left')
train_copy_df.shape

(14993, 29)

In [181]:
print(train_copy_df.loc[train_copy_df['Magnitude'].isnull()].shape)
print(train_copy_df.loc[train_copy_df['Score'].isnull()].shape)

(0, 29)
(0, 29)


In [179]:
train_copy_df.loc[train_copy_df['Magnitude'].isnull(),'Magnitude'] = 0.0
train_copy_df.loc[train_copy_df['Score'].isnull(),'Score'] = 0.0

### Add Description Word Count field

In [154]:
train_copy_df.loc[train_copy_df['Description'].isnull(),'Description'] = ''

In [155]:
countNanInColumn(train_copy_df,'Description')

0

In [156]:
train_copy_df['WordCount'] = train_copy_df['Description'].str.split().apply(lambda x: len(x))

In [158]:
np.sum(train_copy_df['WordCount'].isnull())

0

### Drop Description Field

In [172]:
train_copy_df = train_copy_df.drop('Description',axis=1)

In [180]:
train_copy_df.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,PetID,PhotoAmt,AdoptionSpeed,NumColors,IsMixedBreed,NameFrequency,Magnitude,Score,WordCount
0,2,Nibble,3,299,0,1,1,7,0,1,1,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,86e1089a3,1.0,2,2,0,1,2.4,0.3,69
1,2,No Name Yet,1,265,0,1,1,2,0,2,2,3,3,3,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,6296e909a,2.0,0,2,0,22,0.7,-0.2,23
2,1,Brisco,1,307,0,1,2,7,0,2,2,1,1,2,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,3422e4906,7.0,3,2,0,1,3.7,0.2,69
3,1,Miko,4,307,0,2,1,2,0,2,1,1,1,2,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,5842f1ff5,8.0,2,2,0,11,0.9,0.9,25
4,1,Hunter,1,307,0,1,1,0,0,2,1,2,2,2,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,850a43f90,3.0,2,1,0,5,3.7,0.6,81


### Save to npy

In [183]:
## Reload the the first 10 components of delta dataset
train_sentiment_cleaned = np.save('petfinder_train_sentiment_cleaned.npy',train_copy_df)